## Data Processing

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
bucket = "sand-test-central-481423469601-us-east-1"

In [ ]:
#file = "cleaned_historical_trips_2015_2022.csv"

#s3_data_location = f"s3://{bucket}/{file}*"
#trips = pd.read_csv(s3_data_location, parse_dates = True)

In [ ]:
#trips = trips[(trips["starttime"] > "2017-09-01") & (trips["stoptime"] < "2022-08-31")]

In [ ]:
#trips_start = trips[["starttime", "start station id", "start station name"]]
#trips_stop = trips[["stoptime", "end station id", "end station name"]]

In [ ]:
#trips_start.to_csv("model_trips_start_station_2017_2022.csv")
#trips_stop.to_csv("model_trips_stop_station_2017_2022.csv")

### Trip Start Station

In [ ]:
start_file = "model_trips_start_station_2017_2022.csv"

s3_start_location = f"s3://{bucket}/{start_file}*"
trips_start = pd.read_csv(s3_start_location, parse_dates = True)
trips_start.shape

Check start station id matches up with start station name. In this case, some stations have changed names due to location changes or due to a lack of data standardization. `trips_start_lookup` is a lookup table to match between the different station ids and station names.

In [ ]:
#trips_start["start station id"].nunique()

In [ ]:
#trips_start["start station name"].nunique()

In [ ]:
#trips_start.drop_duplicates(subset = ["start station id", "start station name"]).to_csv("unique_start.csv")

In [ ]:
#trips_start_lookup = trips_start.drop(["Unnamed: 0", "starttime"], axis = 1).drop_duplicates()

In [ ]:
#trips_start_lookup.to_csv("trip_start_station_id_lookup.csv", index = False)

Will use start station id and not start station name. From manually looking at the data, station name has more variation and very similar station names have the same station id.

In [ ]:
trips_start = trips_start.drop(["Unnamed: 0", "start station name"], axis = 1)
trips_start["starttime"] = pd.to_datetime(trips_start["starttime"])
trips_start

In [ ]:
trips_start_group = trips_start.groupby(["starttime", "start station id"], as_index = False).size()
trips_start_group.set_index("starttime", inplace = True)
sum(trips_start_group["size"])

In [ ]:
trip_start_15min = trips_start_group.groupby([pd.Grouper(freq = "15min"), "start station id"])
trip_start_15min = trip_start_15min["size"].sum().unstack("start station id").fillna(0)
sum(trip_start_15min.sum())

In [ ]:
trip_start_15min

Find when the first trip took place for each station. This will be used as a proxy for determining when the station was launched.

In [ ]:
trips_start_min = trips_start.join(trips_start.groupby("start station id")["starttime"].agg(["min", "max"]), on = "start station id")[["start station id", "min"]].drop_duplicates()
trips_start_min

Fill in values for where the station did not exist yet with N/A. If the station was removed, keep the zeroes till the end of the series.

In [ ]:
for station in tqdm(trip_start_15min.columns):
    
    for row in range(len(trip_start_15min[station])):
    
        station_min = trips_start_min[trips_start_min["start station id"] == station]["min"].values[0]
        temp_time = trip_start_15min[station].index[row]

        if temp_time <= station_min and trip_start_15min[station].index[row + 1] < station_min:
            trip_start_15min[station][temp_time] = np.nan

        else:
            break

In [ ]:
sum(trip_start_15min.sum())

In [ ]:
for row in tqdm(range(len(trip_start_15min[436]))):
    
    station_min = trips_start_min[trips_start_min["start station id"] == 436]["min"].values[0]
    temp_time = trip_start_15min[436].index[row]
    
    if temp_time <= station_min and trip_start_15min[436].index[row + 1] < station_min:
        trip_start_15min[436][temp_time] = np.nan
    
    else:
        break

In [ ]:
trip_start_15min[436].sum()

### Trip End Station

In [ ]:
end_file = "model_trips_stop_station_2017_2022.csv"

s3_end_location = f"s3://{bucket}/{end_file}*"
trips_stop = pd.read_csv(s3_end_location, parse_dates = True)